In [10]:
from collections import namedtuple

In [11]:
lines = [line.rstrip('\n') for line in open('dict_new.txt')]

In [12]:
for line in lines[:18]:
    print(line)

!Словарь РуСентиЛекс
! Структура: 
! 1. слово или словосочетание,
! 2. Часть речи или синтаксический тип группы,
! 3. слово или словосочетание в лемматизированной форме, 
! 4. Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная оценка, зависит от контекста (positive/negative),
! 5. Источник: оценка (opinion), чувство (feeling), факт (fact),
! 6. Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.
!
!RuSentiLex Structure
!1. word or phrase,
!2. part of speech or type of syntactic group,
!3. initial word (phrase) in a lemmatized form,
!4. Sentiment: positive, negative, neutral or positive/negative (indefinite, depends on the context),
!5. Source: opinion, feeling (private state), or fact (sentiment connotation),
!6. Ambiguity: if sentiment is different for senses of an ambiguous word, then sentiment orie

In [4]:
lines[18:25]

['аборт, Noun, аборт, negative, fact',
 'абортивный, Adj, абортивный, negative, fact',
 'абракадабра, Noun, абракадабра, negative, opinion',
 'абсурд, Noun, абсурд, negative, opinion',
 'абсурдность, Noun, абсурдность, negative, opinion',
 'абсурдный, Adj, абсурдный, negative, opinion',
 'авантюра, Noun, авантюра, negative, opinion']

In [5]:
len(lines[18:])

16150

In [6]:
list_of_tone = []
names = ['word', 'part_of_speech', 'lemm_word', 'tonality']
for line in lines[18:]:
    list_of_tone.append(dict(zip(names, line.split(', '))))

### Ошибки
естественный, Adj, естественный, neutral, fact, fact, "ПРИРОДНОГО ПРОИСХОЖДЕНИЯ"

за что деньги, VG, PredicativeG, за что деньги, negative, opinion

настроение то, что нужно, NG, настроение то что нужный, positive, feeling

пострадать, , Verb, пострадать, negative, feeling

уровень жизни растет, VG, уровень жизнь, расти, positive, fact

### Нормальные примеры с многозначностью
аморфность, Noun, аморфность, negative, opinion, "БЕЗДЕЯТЕЛЬНЫЙ"

аморфность, Noun, аморфность, negative, opinion, "РАСПЛЫВЧАТЫЙ (НЕЯСНЫЙ)"

аморфность, Noun, аморфность, neutral, fact, "АМОРФНОЕ СТРОЕНИЕ"

аморфный, Adj, аморфный, negative, opinion, "БЕЗДЕЯТЕЛЬНЫЙ"

аморфный, Adj, аморфный, negative, opinion, "РАСПЛЫВЧАТЫЙ (НЕЯСНЫЙ)"

аморфный, Adj, аморфный, neutral, fact, "АМОРФНОЕ СТРОЕНИЕ"

атака, Noun, атака, negative, fact, "АТАКА, ВОЕННЫЙ УДАР"

атака, Noun, атака, positive, fact, "АТАКА (ИГРОВОЕ ДЕЙСТВИЕ)" 

байка, Noun, байка, neutral, fact, "БАЙКОВАЯ ТКАНЬ"

байка, Noun, байка, positive/negative, opinion, "ВЫДУМКА, ВЫМЫСЕЛ"

балласт, Noun, балласт, negative, opinion, "БАЛЛАСТ (БРЕМЯ)"

In [7]:
mistakes = ['PredicativeG', 'NG', 'Verb', 'уровень жизнь']

In [90]:
#example of mistakes
for m in mistakes:
    print(Dict[m])

[features(word='за что деньги', part_of_speech='VG', tonality='за что деньги', source='negative', extra='opinion')]
[features(word='настроение то', part_of_speech='что нужно', tonality='настроение то что нужный', source='positive', extra='feeling'), features(word='покой', part_of_speech='Noun', tonality='positive', source='opinion', extra='')]
[features(word='пострадать', part_of_speech='', tonality='пострадать', source='negative', extra='feeling')]
[features(word='уровень жизни растет', part_of_speech='VG', tonality='расти', source='positive', extra='fact')]


In [8]:
features = namedtuple('features', ['word', 'part_of_speech', 'tonality', 'source', 'extra'])

In [9]:
def get_Dict():
    Dict = {}
    for i, line in enumerate(lines[18:]):
        l = list(map(lambda x: x.strip(), line.split(',')))
        lemm_word, featur, extra = l[2], l[:2] + l[3:5], l[5:]
        if not len(extra):
            featur += ['']
        else:
            featur += ["; ".join(extra).strip('"')]
        if lemm_word not in mistakes:
            prev = Dict.get(lemm_word, [])
            prev.append(features(*featur))
            Dict[lemm_word] = prev
    return Dict

In [10]:
Dict = get_Dict()

In [11]:
len(Dict)

13586

In [12]:
sum(map(lambda x: len(x), Dict.values()))

16145

In [15]:
Dict['басня']

[features(word='басня', part_of_speech='Noun', tonality='negative', source='opinion', extra='ВЫДУМКА; ВЫМЫСЕЛ'),
 features(word='басня', part_of_speech='Noun', tonality='neutral', source='fact', extra='БАСНЯ (ПРОИЗВЕДЕНИЕ)')]

In [16]:
Dict['безоблачный']

[features(word='безоблачный', part_of_speech='Adj', tonality='positive', source='fact', extra='БЕЗОБЛАЧНАЯ ПОГОДА'),
 features(word='безоблачный', part_of_speech='Adj', tonality='positive', source='opinion', extra='ИДИЛЛИЯ; БЕЗМЯТЕЖНОЕ СУЩЕСТВОВАНИЕ')]

In [9]:
Dict['аборт'][0].word

'аборт'

In [17]:
import pickle
with open('dict.pickle', 'wb') as f:
    pickle.dump(Dict, f)

In [6]:
import pickle
from collections import namedtuple
features = namedtuple('features', ['word', 'part_of_speech', 'tonality', 'source', 'extra'])
with open('dict.pickle', 'rb') as f:
    Dict = pickle.load(f)